## 텍스트 마이닝

텍스트 마이닝 목표
공모전 분야(category): 각 분야를 리스트 형태로 정리.
대상(참여 조건): 텍스트에서 대상 관련 정보를 추출.

In [5]:
import pandas as pd
import re

# 1. JSON 파일 로드
df = pd.read_json('./competitions.json')

# 2. 공모전 분야(category) 추출 함수
def extract_categories(text):
    try:
        # "분야 :" 다음의 텍스트를 추출
        category_pattern = re.compile(r'분야\s*:\s*(.*)')
        match = category_pattern.search(text)
        if match:
            categories = match.group(1).split(',')  # 쉼표로 나누기
            return [cat.strip() for cat in categories]  # 공백 제거
        return None
    except:
        return None

df['분야'] = df['category'].apply(extract_categories)

# 3. 대상(참여 조건) 추출 함수
def extract_target(text):
    if text and isinstance(text, str):  # 비어있지 않은 문자열인지 확인
        return text.strip()  # 공백 제거 후 반환
    return None


df['대상'] = df['target'].apply(extract_target)

# 4. 데이터 출력 확인
print(df[['title', '분야', '대상']].head())


                                               title  \
0           청주오스코 슬로건 아이디어 & MICE 행사 기획 공모전신규SPECIAL   
1                   컴투스 글로벌 게임개발 공모전 컴:온 2024SPECIAL   
2                     [타임금융교육원] 금융실무과정 88기 모집SPECIAL   
3  [무료] 2025 삼성 Financial Networks 금융연수프로그램 7기 모집...   
4  2025 제3회 ChunMan Art for Young 공모전 (천만 아트 포 영 ...   

                                              분야                 대상  
0  [기획/아이디어, 광고/마케팅, 논문/리포트, 네이밍/슬로건, 대외활동/서포터즈]               제한없음  
1         [웹/모바일/IT, 게임/소프트웨어, 과학/공학, 대외활동/서포터즈]  일반인, 대학생, 청소년, 기타  
2      [광고/마케팅, 예체능/미술/음악, 대외활동/서포터즈, 취업/창업, 기타]       일반인, 대학생, 기타  
3        [기획/아이디어, 광고/마케팅, 대외활동/서포터즈, 취업/창업, 기타]       일반인, 대학생, 기타  
4           [영상/UCC/사진, 웹/모바일/IT, 예체능/미술/음악, 기타]       일반인, 대학생, 기타  


In [79]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# 1. 공모전 데이터 로드
gong_info = pd.read_json('./competitions.json')  # 공모전 상세 정보

# 'gong_id' 열 추가: 공모전 데이터에 고유 ID 추가
gong_info['gong_id'] = gong_info.index

# 2. 사용자-공모전 상호작용 데이터 생성 (가상 데이터 생성 함수)
def generate_interaction_data(gong_info, num_users=100):
    """
    공모전 데이터를 기반으로 가상 사용자-공모전 상호작용 데이터를 생성합니다.
    """
    gong_ids = range(len(gong_info))  # 공모전 ID

    interaction_data = []
    for user_id in range(1, num_users + 1):
        interacted = np.random.choice(gong_ids, size=np.random.randint(1, 5), replace=False)
        for gong_id in interacted:
            interaction_data.append({'user_id': user_id, 'gong_id': gong_id, 'gong_rank': np.random.randint(1, 11)})
    return pd.DataFrame(interaction_data)

# 3. 사용자-공모전 상호작용 데이터 로드 또는 생성
try:
    user_info = pd.read_csv('./data/user_info.csv')  # 실제 데이터가 있는 경우
except FileNotFoundError:
    user_info = generate_interaction_data(gong_info)  # 데이터가 없는 경우 가상 데이터 생성

# 4. 사용자-공모전 평점 매트릭스 생성
user_gong_rating = user_info.pivot_table('gong_rank', index='user_id', columns='gong_id').fillna(0)
user_gong_matrix = user_gong_rating.values.T  # 행렬 전치 (공모전명 × 사용자)

# 5. SVD 적용
SVD = TruncatedSVD(n_components=12)  # 주성분 12개 추출
svd_matrix = SVD.fit_transform(user_gong_matrix)
correlation_matrix = np.corrcoef(svd_matrix)

def gongmo_recommend(title, category, target, threshold=0.8):
    """
    SVD를 이용하여 공모전을 추천하는 함수
    :param title: 기준 공모전명
    :param category: 추천받고자 하는 공모전의 분야
    :param target: 추천받고자 하는 참여 대상 조건
    :param threshold: 유사도 임계값
    :return: 추천 공모전 리스트
    """
    # 기준 공모전 인덱스 찾기
    try:
        title_index = gong_info[gong_info['title'] == title].index[0]
    except IndexError:
        print(f"Title '{title}' not found in the competition data.")
        return []

    # 조건(category, target)이 주어진 경우 필터링
    filtered_gong = gong_info.copy()
    if category or target:
        filtered_gong = filtered_gong[
            (filtered_gong['category'].str.contains(category, na=False)) &
            (filtered_gong['target'].str.contains(target, na=False))
        ]
    
    # 사용자-공모전 평점 매트릭스 생성
    user_data = user_info.copy()
    gong_data = filtered_gong[['gong_id', 'title']].copy()
    user_gong_data = pd.merge(user_data, gong_data, on='gong_id')

    user_gong_rating = user_gong_data.pivot_table('gong_rank', index='user_id', columns='title').fillna(0)
    user_gong_matrix = user_gong_rating.values.T

    # SVD 적용
    SVD = TruncatedSVD(n_components=12)
    svd_matrix = SVD.fit_transform(user_gong_matrix)

    # 공모전 간의 상관계수 계산
    correlation_matrix = np.corrcoef(svd_matrix)

    # 기준 공모전과 유사한 공모전 탐색
    similar_scores = correlation_matrix[title_index]

    # 유사도 임계값 이상인 공모전 찾기
    recommended_indices = [i for i, score in enumerate(similar_scores) if score >= threshold and i != title_index]
    recommended_titles = gong_info.iloc[recommended_indices]['title'].tolist()

    return recommended_titles

# 추천 예시
recommendations = gongmo_recommend(
    title="청주오스코 슬로건 아이디어 & MICE 행사 기획 공모전신규SPECIAL",
    category="기획/아이디어",
    target="제한없음",
    threshold=0.5  # 유사도 임계값 조정
)
print("추천 공모전:", recommendations)


추천 공모전: []


In [80]:
title_index = gong_info[gong_info['title'] == "청주오스코 슬로건 아이디어 & MICE 행사 기획 공모전신규SPECIAL"].index[0]
similar_scores = correlation_matrix[title_index]

# 유사도가 높은 순으로 출력
similar_titles = pd.DataFrame({
    '공모전명': gong_info['title'],
    '유사도': similar_scores
}).sort_values(by='유사도', ascending=False)

print(similar_titles.head())  # 가장 유사한 공모전 상위 5개 출력


                                                 공모전명       유사도
0            청주오스코 슬로건 아이디어 & MICE 행사 기획 공모전신규SPECIAL  1.000000
25                         2024 글로벌 인플루언서 엑스포 서포터즈 모집  0.731912
8   [무료] 25년 엠금융서비스의 MFE (M Finance Expert) (현직자 멘...  0.718507
26                         신세계백화점 커뮤니티 대학생 서포터즈 2기 모집  0.615999
30                               성동구 2024 아동권리 그림 공모전  0.608932


In [105]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings('ignore')

# 1. 공모전 데이터 로드
gong_info = pd.read_json('./competitions.json')  # 공모전 상세 정보

# 2. 공모전의 카테고리와 제목을 결합하여 텍스트 데이터를 만들기
gong_info['content'] = gong_info['category'] + " " + gong_info['title']

# 3. 텍스트 데이터에서 TF-IDF 벡터 생성
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(gong_info['content'])

# 4. 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 5. 공모전 추천 함수 (유사도 값 함께 출력)
def content_based_recommendation(title, top_n=5):
    """
    주어진 공모전 제목을 기반으로 가장 유사한 공모전을 추천합니다.
    :param title: 기준 공모전 제목
    :param top_n: 추천할 공모전의 수
    :return: 추천 공모전 제목 리스트와 유사도 값
    """
    # 공모전 제목을 기준으로 해당 공모전 인덱스 찾기
    idx = gong_info[gong_info['title'] == title].index[0]
    
    # 해당 공모전과 다른 공모전들 간의 유사도 계산
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도 높은 순으로 정렬하고, 자기 자신 제외
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:]
    
    # 상위 N개 추천 공모전 제목과 유사도 값 반환
    recommended_indices = [i[0] for i in sim_scores[:top_n]]
    recommended_scores = [i[1] for i in sim_scores[:top_n]]
    recommended_titles = gong_info['title'].iloc[recommended_indices].tolist()
    
    return list(zip(recommended_titles, recommended_scores))

# 6. 추천 예시
recommendations = content_based_recommendation(
    title="2024년 대한민국 청년정책 어워즈 청년정책 발굴단 모집",  # 공모전 제목만 입력
    top_n=5  # 추천할 공모전 수
)

# 추천 공모전 출력
if recommendations:
    print("추천 공모전 상위 5개 (유사도 포함):")
    for idx, (rec, score) in enumerate(recommendations):
        print(f"{idx+1}. {rec} - 유사도: {score:.4f}")
else:
    print("추천할 공모전이 없습니다.")


추천 공모전 상위 5개 (유사도 포함):
1. 2024 글로벌 인플루언서 엑스포 서포터즈 모집 - 유사도: 0.1771
2. 신세계백화점 커뮤니티 대학생 서포터즈 2기 모집 - 유사도: 0.1651
3. 제2기 문화체육관광부 2030자문단 단원 모집 - 유사도: 0.1446
4. 제2기 개인정보보호위원회 2030 자문단 모집 - 유사도: 0.1446
5. 2024년 물류데이터 활용 신규 서비스 공모전 - 유사도: 0.1073
